In [1]:
import pandas as pd
import numpy as np

In [2]:
matches = pd.read_csv('/Users/tliu/Desktop/Erdos Project/Data/matches.csv')

In [3]:
matches.head()

,player1,player2,score1,score2,best_of,tournament_id,date,year
0,Colin Roscoe,Jackie Rea,9,6,17,753,NaN,1982
1,Tommy Murphy,Clive Everton,9,4,17,753,NaN,1982
2,Vic Harris,Marcus Owen,9,4,17,753,NaN,1982
3,Bob Harris,Graham Cripsey,9,6,17,753,NaN,1982
4,Geoff Foulds,Matt Gibson,9,3,17,753,NaN,1982


In [4]:
tournaments = list(matches.tournament_id.unique())


In [5]:
matches[matches['tournament_id'] == tournaments[1040]]

,player1,player2,score1,score2,best_of,tournament_id,date,year
111102,Xing Zihao,Fergal O'Brien,4,3,7,5848,2023-11-02,2023
111103,Cao Yupeng,David Lilley,4,2,7,5848,2023-10-30,2023
111104,Zhou Yuelong,Anthony Hamilton,4,0,7,5848,2023-10-30,2023
111105,Kyren Wilson,Mostafa Dorgham,4,2,7,5848,2023-12-11,2023
111106,Gary Wilson,Elliot Slessor,4,3,7,5848,2023-12-11,2023
...,...,...,...,...,...,...,...,...
111222,Noppon Saengkham,Sanderson Lam,5,3,9,5848,2023-12-15,2023
111223,John Higgins,Tom Ford,5,3,9,5848,2023-12-15,2023
111224,Gary Wilson,Zhou Yuelong,6,5,11,5848,2023-12-16,2023
111225,Noppon Saengkham,John Higgins,6,3,11,5848,2023-12-16,2023


In [6]:
#Using matches history before tornament xxxx to predict the results of tournament xxxx
from elo_v3 import elo_rating
from sklearn.metrics import mean_squared_error


#Ks represent K-factors we use
#nth means nth tournament among 1085 tournaments
Ks = [4, 6, 8, 15, 20]
nth = [750, 999, 1040, 1080]
scores = np.zeros((5,4))


i=0
for K in Ks:
    j=0
    for n in nth:
        past_tournaments = tournaments[:n]
        current = tournaments[n]
        past_index = matches['tournament_id'].map(lambda x: x in past_tournaments)
        

        matches_past = matches[past_index]
        matches_future = matches[matches['tournament_id'] == current]

        #Update statistics of players
        elo = elo_rating()
        elo.update_stats(matches = matches_past, K_factor = K, ratio = 0.1)

        #Store the actual result in y
        matches_future['result'] = 0  
        y = np.array(matches_future['result'])

        #Compute expected win_rate
        X = matches_future[['player1', 'player2', 'best_of']]
        _, expected = elo.predict(X)
        print(expected)

        #Compute the actual frame_rate
        actual = np.array(matches_future['score1']/(matches_future['score1']+matches_future['score2']))
        print(actual)

        #Record the accuracy score
        from sklearn.metrics import accuracy_score
        scores[i,j] = mean_squared_error(expected, actual)
        
        j += 1
    i+=1




/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/2098003915.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


[0.40914525 0.48001066 0.78329968 0.61004467 0.52622591 0.77902611
 0.38461624 0.56525075 0.8049462  0.19270945 0.8896823  0.17544764
 0.83889105 0.89934791 0.24277966 0.88795296 0.59025025 0.42372508
 0.81645321 0.58722369 0.93583612 0.24048908 0.39651675 0.6921095
 0.29317778 0.37460765 0.79616571 0.67480527 0.69264198 0.5168686
 0.87814715 0.5799334  0.55416165 0.239577   0.63760865 0.75676045
 0.87626214 0.73399752 0.79248994 0.88313927 0.83166869 0.626563
 0.29943286 0.60885457 0.71960437 0.46319171 0.81794705 0.35491586
 0.61360708 0.47751518 0.63818611 0.56340679 0.86442036 0.32027681
 0.7320405  0.24927176 0.64106741 0.51374653 0.53867261 0.83026408
 0.239577   0.35606144 0.84455265 0.65135486 0.67863372 0.81191475
 0.66707828 0.8519528  0.29317778 0.66429644 0.81115001 0.51998934
 0.8223718  0.75905399 0.8988944  0.47813895 0.77816418 0.30895699
 0.63529485 0.60408132 0.77729986 0.7407749  0.5926666  0.77381857
 0.7879312  0.27289178 0.61833804 0.65982175 0.18168098 0.74696679

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/2098003915.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


[0.35377194 0.65418811 0.70578503 0.42311474 0.75676045 0.66984852
 0.77206355 0.73643093 0.37812836 0.81943141 0.72161767 0.7422125
 0.58782954 0.59145898 0.52934125 0.93658263 0.47751518 0.44090253
 0.6165665  0.49562511 0.68568011 0.64164245 0.74507206 0.88442316
 0.52061331 0.89866696 0.77643314 0.4037173  0.54177737 0.42128514
 0.87380181 0.3062946  0.60228618 0.78117041 0.50312496 0.24277966
 0.37636637 0.77945617 0.61242094 0.34921308 0.85005086 0.58964548
 0.45078487 0.49437524 0.4329071  0.49687504 0.48001066 0.85875649
 0.76224005 0.7109495  0.41095957 0.58419052 0.78583498 0.42189477
 0.7378841  0.64221709 0.66984852 0.5093739  0.81229622 0.46132739
 0.82885017 0.88569476 0.33403307 0.76852478 0.42984097 0.69899148
 0.59447593 0.5670929  0.56032964 0.87653295 0.56340679 0.2630841
 0.58964548 0.52622591 0.49875    0.9022519  0.54549879 0.91548307
 0.39891212 0.66818777 0.61004467 0.32355107 0.63413559 0.71351147
 0.32849497 0.28597777 0.22140455 0.90114393 0.80178688 0.726114

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/2098003915.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


...

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/2098003915.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


[0.33961733 0.66262184 0.60706699 0.94374707 0.55786457 0.78202392
 0.50687457 0.5093739  0.78287503 0.81943141 0.22270014 0.90593657
 0.96417062 0.56279174 0.9234378  0.74269055 0.86875553 0.91945759
 0.71351147 0.94144779 0.95997873 0.93292416 0.13528691 0.91133134
 0.78329968 0.72561752 0.88288101 0.95324719 0.96809292 0.61063923
 0.78709451 0.38225213 0.44398611 0.53929381 0.27041856 0.78834863
 0.25445339 0.6775423  0.1974197  0.68621867 0.75119578 0.56279174
 0.92001126 0.69846522 0.75398872 0.88720459 0.48438008 0.85320966
 0.56647906 0.88339703 0.45698162 0.55971365 0.24323955 0.24601128
 0.26649088 0.76629364 0.9156763  0.71351147 0.51312199 0.86294871
 0.92019504 0.66038267 0.92757351 0.55601389 0.84586097 0.24973989
 0.80612106 0.55292599 0.72859384 0.80612106 0.75537724 0.63760865
 0.39532092 0.68297985 0.39532092 0.61538376 0.95154724 0.85936187
 0.55292599 0.29162601 0.34355313 0.82813971 0.68783133 0.1424608
 0.50624967 0.80651149 0.50312496 0.41763246 0.21500771 0.39532

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)


[0.75537724 0.68943945 0.61951732 0.84909236 0.7768668  0.77859545
 0.74364489 0.64164245 0.83821414 0.50812428 0.8241904  0.68352089
 0.68621867 0.35663485 0.69157651 0.72211961 0.88000737 0.49750002
 0.60885457 0.6165665  0.82742692 0.54239794 0.51811707 0.59326999
 0.44707401 0.87380181 0.7488522  0.87571899 0.47003595 0.58297545
 0.36066016 0.84716026 0.36935147 0.43843893 0.7369159  0.85845299
 0.86383331 0.43967036 0.6479408  0.79494586 0.60288487 0.81419467
 0.34242639 0.72661188 0.81038289 0.41459543 0.22970105 0.71300016
 0.63529485 0.58782954 0.46692338 0.60228618 0.59868766 0.8025803
 0.50687457 0.78959734 0.61004467 0.84323529 0.42250463 0.52809537
 0.43168002 0.58843512 0.30364507 0.53307662 0.64164245 0.42128514
 0.72760399 0.15348826 0.64049195 0.53494295 0.6159753  0.57871486
 0.86442036 0.75445215 0.80337133 0.37168381 0.46257015 0.3840247
 0.6479408  0.81191475 0.80218389 0.51811707 0.77816418 0.58297545
 0.86324411 0.3520592  0.6165665  0.81457258 0.34130143 0.320276

/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/2098003915.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


In [7]:
print(scores)

[[0.04511027 0.0497169  0.07126358 0.03380794]
 [0.04851583 0.05204659 0.07118478 0.03167362]
 [0.05098815 0.0539865  0.07163361 0.03100015]
 [0.05719899 0.05869029 0.07438659 0.03349601]
 [0.06065392 0.06133523 0.07678993 0.0366705 ]]


Ks = [4, 6, 8, 15, 20]
nth = [750, 999, 1040, 1080]

[[0.04511027 0.0497169  0.07126358 0.03380794]
 [0.04851583 0.05204659 0.07118478 0.03167362]
 [0.05098815 0.0539865  0.07163361 0.03100015]
 [0.05719899 0.05869029 0.07438659 0.03349601]
 [0.06065392 0.06133523 0.07678993 0.0366705 ]]

In [8]:
#Using matches history before tornament xxxx to predict the results of tournament xxxx
from elo_v3 import elo_rating
from sklearn.metrics import mean_squared_error


#Ks represent K-factors we use
#nth means nth tournament among 1085 tournaments
Ks = [2,3,5,7,8]
nth = [750, 999, 1040, 1080]
scores = np.zeros((5,4))


i=0
for K in Ks:
    j=0
    for n in nth:
        past_tournaments = tournaments[:n]
        current = tournaments[n]
        past_index = matches['tournament_id'].map(lambda x: x in past_tournaments)
        

        matches_past = matches[past_index]
        matches_future = matches[matches['tournament_id'] == current]

        #Update statistics of players
        elo = elo_rating()
        elo.update_stats(matches = matches_past, K_factor = K, ratio = 0.1)

        #Store the actual result in y
        matches_future['result'] = 0  
        y = np.array(matches_future['result'])

        #Compute expected win_rate
        X = matches_future[['player1', 'player2', 'best_of']]
        _, expected = elo.predict(X)
        print(expected)

        #Compute the actual frame_rate
        actual = np.array(matches_future['score1']/(matches_future['score1']+matches_future['score2']))
        print(actual)

        #Record the accuracy score
        from sklearn.metrics import accuracy_score
        scores[i,j] = mean_squared_error(expected, actual)
        
        j += 1
    i+=1




/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/711043284.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


[0.44028635 0.50249998 0.7558389  0.62714777 0.48001066 0.68783133
 0.41763246 0.58661758 0.73154982 0.26697985 0.80999843 0.24554785
 0.76539701 0.84157582 0.31378133 0.81457258 0.54859611 0.44954729
 0.74173388 0.55724785 0.89425874 0.29890869 0.43967036 0.65531837
 0.34637767 0.38048268 0.72262098 0.65248946 0.65135486 0.5106234
 0.81038289 0.55107127 0.56831998 0.33849684 0.6159753  0.68675673
 0.8025803  0.67644893 0.71300016 0.8547683  0.74932208 0.61715736
 0.36354746 0.57260806 0.68890392 0.43597836 0.75859646 0.4139888
 0.56340679 0.46754569 0.58904043 0.50999867 0.78667527 0.39293301
 0.66038267 0.3208213  0.57688526 0.47439741 0.55168967 0.75398872
 0.2719008  0.42250463 0.76718789 0.58904043 0.66984852 0.71504211
 0.6212836  0.7904268  0.37285223 0.67972319 0.78031449 0.50749944
 0.77294226 0.7047457  0.86960831 0.4893766  0.7407749  0.32027681
 0.62480647 0.565865   0.75629997 0.67535364 0.54673815 0.7640476
 0.73056676 0.31431989 0.56279174 0.61419964 0.25874532 0.7201085

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/711043284.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


[0.41884904 0.6165665  0.65362233 0.42800387 0.71197592 0.64106741
 0.73981361 0.68081071 0.42128514 0.76718789 0.68943945 0.65078692
 0.54549879 0.5149955  0.50812428 0.90378446 0.47315085 0.43905455
 0.64565631 0.5        0.59447593 0.59568073 0.70785727 0.86026588
 0.49375033 0.85538789 0.7109495  0.45512111 0.52871836 0.46319171
 0.87734233 0.3262929  0.53307662 0.75629997 0.500625   0.32464636
 0.42922839 0.70578503 0.61419964 0.4037173  0.82053848 0.59748577
 0.4831314  0.47190463 0.43229345 0.55786457 0.44769209 0.83889105
 0.76629364 0.65475346 0.44769209 0.59928816 0.76539701 0.42067575
 0.7251195  0.64164245 0.59085475 0.50687457 0.78031449 0.46754569
 0.76584563 0.83479513 0.35262969 0.72908792 0.43536371 0.70318271
 0.62069519 0.59447593 0.51998934 0.8566205  0.60766317 0.24880422
 0.50874911 0.47502081 0.49000133 0.87544664 0.61301418 0.90443467
 0.43536371 0.66818777 0.55416165 0.3208213  0.64622806 0.6846015
 0.35148913 0.31431989 0.22618143 0.87157983 0.78583498 0.72462

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/711043284.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


...

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/711043284.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


[0.33125818 0.64336515 0.5860112  0.9156763  0.56094545 0.70109131
 0.50437489 0.39651675 0.7640476  0.70889017 0.37695334 0.86146364
 0.93021522 0.57932425 0.86960831 0.60647049 0.81038289 0.82742692
 0.76763412 0.91945759 0.9162536  0.87185939 0.16043433 0.83582688
 0.72010853 0.71402223 0.72161767 0.91411915 0.9284088  0.57810523
 0.62010643 0.37812836 0.41702455 0.50374993 0.29890869 0.74743902
 0.30417394 0.57810523 0.14063813 0.55971365 0.76224005 0.5962827
 0.89235257 0.56647906 0.64106741 0.86789802 0.50999867 0.78414719
 0.61479187 0.83685344 0.42678023 0.58115096 0.3078905  0.32904667
 0.23460299 0.71605979 0.88695417 0.72111518 0.50749944 0.7698563
 0.90356684 0.65701046 0.80922774 0.65475346 0.81343708 0.2748805
 0.74029454 0.50187499 0.70682222 0.74507206 0.75026011 0.54053584
 0.43659321 0.67260702 0.3362613  0.60048827 0.92431689 0.82778361
 0.47502081 0.22530751 0.40793707 0.86500529 0.61951732 0.28444878
 0.57199613 0.69104301 0.42250463 0.49187572 0.2621159  0.3618138

/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)


[0.62714777 0.55663096 0.57566435 0.83956567 0.72908792 0.70682222
 0.6846015  0.58115096 0.80019283 0.44522076 0.69793843 0.63064853
 0.58297545 0.49437524 0.70630389 0.62069519 0.78202392 0.54425886
 0.52809537 0.70785727 0.78329968 0.60766317 0.45822263 0.57871486
 0.51936531 0.73105858 0.61892785 0.81191475 0.46754569 0.56340679
 0.45822263 0.7349726  0.36876934 0.36239135 0.75905399 0.84290453
 0.82599433 0.40191314 0.63529485 0.764498   0.58175938 0.68406145
 0.53805128 0.70940582 0.69104301 0.45884333 0.37051689 0.64336515
 0.62245933 0.47876279 0.52622591 0.68135373 0.49687504 0.73885001
 0.55045271 0.59447593 0.64908052 0.81076675 0.37051689 0.53494295
 0.48375572 0.64336515 0.3307046  0.49562511 0.70889017 0.33737816
 0.73350912 0.18770378 0.60467908 0.51749286 0.71043548 0.57077161
 0.79535308 0.79535308 0.78372374 0.31163212 0.43843893 0.41580948
 0.56831998 0.6802672  0.70056714 0.64508414 0.7320405  0.549834
 0.8241904  0.34921308 0.55292599 0.76896922 0.44769209 0.295774

/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1958/711043284.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


In [9]:
print(scores)

[[0.04081901 0.04758738 0.07280416 0.03812379]
 [0.042967   0.04850241 0.07176707 0.035694  ]
 [0.04693483 0.05093644 0.07112929 0.03248546]
 [0.04983187 0.05306772 0.07137247 0.03119329]
 [0.05098815 0.0539865  0.07163361 0.03100015]]


Ks = [2,3,5,7,8]
nth = [750, 999, 1040, 1080]

Ks = [4, 6, 8, 15, 20]
nth = [750, 999, 1040, 1080]

[[0.04081901 0.04758738 0.07280416 0.03812379]
 [0.042967   0.04850241 0.07176707 0.035694  ]
 [0.04693483 0.05093644 0.07112929 0.03248546]
 [0.04983187 0.05306772 0.07137247 0.03119329]
 [0.05098815 0.0539865  0.07163361 0.03100015]]

 [[0.04511027 0.0497169  0.07126358 0.03380794]
 [0.04851583 0.05204659 0.07118478 0.03167362]
 [0.05098815 0.0539865  0.07163361 0.03100015]
 [0.05719899 0.05869029 0.07438659 0.03349601]
 [0.06065392 0.06133523 0.07678993 0.0366705 ]]

In [14]:
scores2 = np.array(
[[0.04511027, 0.0497169,  0.07126358, 0.03380794],
 [0.04851583, 0.05204659, 0.07118478, 0.03167362],
 [0.05098815, 0.0539865,  0.07163361, 0.03100015],
 [0.05719899, 0.05869029, 0.07438659, 0.03349601],
 [0.06065392, 0.06133523, 0.07678993, 0.0366705 ]]
 )

In [41]:
scores1 = scores
scores1

array([[0.04081901, 0.04758738, 0.07280416, 0.03812379],
       [0.042967  , 0.04850241, 0.07176707, 0.035694  ],
       [0.04693483, 0.05093644, 0.07112929, 0.03248546],
       [0.04983187, 0.05306772, 0.07137247, 0.03119329],
       [0.05098815, 0.0539865 , 0.07163361, 0.03100015]])

In [42]:
scores_df = pd.concat([pd.DataFrame(scores1), pd.DataFrame(scores2)])

In [43]:
scores_df['K-factor'] = [2,3,5,7,8, 4, 6, 8, 15, 20]

In [44]:
scores_df.set_index('K-factor', inplace=True)

In [46]:
scores_df.rename(columns = {
    0: 750,
    1: 999,
    2: 1040,
    3: 1080
})

,750,999,1040,1080
K-factor,,,,
2,0.040819,0.047587,0.072804,0.038124
3,0.042967,0.048502,0.071767,0.035694
5,0.046935,0.050936,0.071129,0.032485
7,0.049832,0.053068,0.071372,0.031193
8,0.050988,0.053987,0.071634,0.031000
4,0.045110,0.049717,0.071264,0.033808
6,0.048516,0.052047,0.071185,0.031674
8,0.050988,0.053986,0.071634,0.031000
15,0.057199,0.058690,0.074387,0.033496


In [47]:
average_score = scores_df.mean(axis=1)
average_score

K-factor
2     0.049834
3     0.049733
5     0.050372
7     0.051366
8     0.051902
4     0.049975
6     0.050855
8     0.051902
15    0.055943
20    0.058862
dtype: float64